```bash
# Virtuelle Umgebung erstellen
python -m venv hoppe-env

# Umgebung aktivieren
source hoppe-env/Scripts/activate

# Pakete installiereb
pip install -r requirements.txt --quiet
```

In [ ]:
import os
import sys
import logging
from datetime import datetime, timedelta
import fabric  # Fabric-spezifische Bibliothek

In [ ]:
# Hauptordner erstellen
os.makedirs('./data', exist_ok=True)

# Unterordner erstellen
for sub_dir in ['raw_data', 'transformed_data', 'gaps_data']:
    os.makedirs(os.path.join('../data', sub_dir), exist_ok=True)

In [ ]:
# Logging einrichten
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("hoppe_etl")

In [ ]:
# Hilfsfunktion für Fabric Storage-Integration
def setup_fabric_storage():
    """
    Richtet die Verbindung zum Fabric Lakehouse ein
    """
    # Annahme: In Fabric ist das Lakehouse bereits konfiguriert
    try:
        # Pfade für die verschiedenen Datenebenen
        lakehouse_path = fabric.get_lakehouse_path()
        raw_path = f"{lakehouse_path}/Files/raw_data"
        transformed_path = f"{lakehouse_path}/Files/transformed_data"
        gaps_path = f"{lakehouse_path}/Files/gaps_data"
        
        # Stellen Sie sicher, dass die Verzeichnisse existieren
        os.makedirs(raw_path, exist_ok=True)
        os.makedirs(transformed_path, exist_ok=True)
        os.makedirs(gaps_path, exist_ok=True)
        
        logger.info(f"Fabric storage paths configured: {raw_path}, {transformed_path}, {gaps_path}")
        
        return {
            'RAW_PATH': raw_path,
            'TRANSFORMED_PATH': transformed_path,
            'GAPS_PATH': gaps_path
        }
    except Exception as e:
        logger.error(f"Failed to setup Fabric storage: {str(e)}")
        raise

In [ ]:
# Hilfsfunktion für Fabric Secrets Management
def get_fabric_secrets():
    """
    Holt Secrets aus dem Fabric KeyVault
    """
    try:
        # Annahme: In Fabric sind die Geheimnisse als Parameter gespeichert
        api_key = fabric.get_secret("HOPPE_API_KEY")
        db_connection = fabric.get_secret("MSSQL_CONNECTION_STRING") 
        
        if not api_key:
            raise ValueError("HOPPE_API_KEY not found in Fabric secrets")
            
        logger.info("Successfully retrieved secrets from Fabric")
        
        return {
            'HOPPE_API_KEY': api_key,
            'MSSQL_CONNECTION_STRING': db_connection
        }
    except Exception as e:
        logger.error(f"Failed to get Fabric secrets: {str(e)}")
        raise


In [ ]:
# Importieren der ETL-Pipeline-Klassen
from config import Config
from pipeline import Pipeline

In [ ]:
def run_fabric_pipeline(mode="all"):
    """
    Führt die ETL-Pipeline in Fabric aus
    
    Args:
        mode (str): 'all', 'timeseries', oder 'fleet'
    """
    try:
        # Fabric Storage einrichten
        storage_config = setup_fabric_storage()
        
        # Secrets holen
        secrets = get_fabric_secrets()
        
        # API-Key überprüfen
        api_key = secrets.get('HOPPE_API_KEY')
        if not api_key:
            raise ValueError("HOPPE_API_KEY not found in environment or secrets")
        
        # Pipeline konfigurieren
        config = Config(
            base_url=os.getenv('HOPPE_BASE_URL', "https://api.hoppe-sts.com/"),
            raw_path=storage_config.get('RAW_PATH', "./data/raw_data"),
            transformed_path=storage_config.get('TRANSFORMED_PATH', "./data/transformed_data"),
            gaps_path=storage_config.get('GAPS_PATH', "./data/gaps_data"),
            max_workers=int(os.getenv('MAX_WORKERS', "8")),
            retry_attempts=int(os.getenv('RETRY_ATTEMPTS', "5")),
            timeout=int(os.getenv('TIMEOUT', "45")),
            days_to_keep=int(os.getenv('DAYS_TO_KEEP', "90")),
            history_days=int(os.getenv('HISTORY_DAYS', "5"))
        )
        
        # Pipeline erstellen und ausführen
        pipeline = Pipeline(config, api_key)
        run_timestamp = pipeline.run(mode)
        
        # Wenn Datenbankverbindung konfiguriert ist, in Datenbank speichern
        db_connection_string = secrets.get('MSSQL_CONNECTION_STRING')
        if db_connection_string:
            try:
                import sqlalchemy as sa
                engine = sa.create_engine(db_connection_string)
                logger.info("Database connection established")
                
                # Daten verarbeiten und in Datenbank speichern
                pipeline.process_and_store_to_db(engine, run_timestamp)
                logger.info("Database integration completed successfully")
            except Exception as e:
                logger.error(f"Database integration failed: {str(e)}")
        else:
            logger.warning("MSSQL_CONNECTION_STRING not set, skipping database integration")
        
        # Protokollieren des Abschlusses für Fabric-Aktivitätsüberwachung
        fabric.log_activity(f"Pipeline run completed successfully with mode: {mode}")
        
        return {"status": "success", "run_timestamp": run_timestamp}
        
    except Exception as e:
        error_msg = f"Pipeline failed: {str(e)}"
        logger.error(error_msg)
        fabric.log_activity(f"Pipeline run failed: {error_msg}", status="error")
        
        return {"status": "error", "error": error_msg}


In [ ]:
# Täglich ausführen (vollständige Pipeline)

# Diese Zelle wird als geplante tägliche Aktivität in Fabric konfiguriert
# Sie führt die vollständige Pipeline aus

result = run_fabric_pipeline(mode="all")
print(f"Pipeline result: {result}")

In [ ]:
# Timeseries Pipeline (alle 15 Minuten)

# Diese Zelle wird als geplante 15-Minuten-Aktivität in Fabric konfiguriert
# Sie führt nur den Timeseries-Teil der Pipeline aus

result = run_fabric_pipeline(mode="timeseries")
print(f"Timeseries pipeline result: {result}")
